In [ ]:
import re
import sys
import pathlib
import functools
import traceback

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import imageio
import img2pdf

import IPython.display

import pydicom

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys
from prototyping import *

In [ ]:
percent_deviation = 2
mm_dist_threshold = 0.5

logfile_directory = pathlib.Path(r'D:\LinacLogFiles')
monaco_directory = pathlib.Path(r'\\monacoda\FocalData\RCCC\1~Clinical')
output_directory = pathlib.Path(r'S:\Physics\Patient Specific Logfile Fluence')

pdf_directory = pathlib.Path(r'P:\Scanned Documents\RT\PhysChecks\Logfile PDFs')

In [ ]:
extra_patient_ids_to_check = []

In [ ]:
with pymedphys.mosaiq.connect('msqsql') as cursor:
    qcls = get_incomplete_qcls(cursor, 'Physics_Check')

In [ ]:
qcls

In [ ]:
patient_ids = extra_patient_ids_to_check + list(qcls['patient_id'])
patient_ids

In [ ]:
logfile_paths = {}

for patient_id in patient_ids:
    paths = list(logfile_directory.joinpath('indexed').glob(f'*/{patient_id}_*/*/*/*/*.trf'))
    if paths:
        logfile_paths[patient_id] = paths

In [ ]:
logfile_paths

In [ ]:
patiend_ids_with_logfiles = list(logfile_paths.keys())

In [ ]:
patient_id = patiend_ids_with_logfiles[0]

In [ ]:
monaco_approval_comments = {}

for patient_id in patiend_ids_with_logfiles:
    paths = list(monaco_directory.glob(f'*~{patient_id}/plan/*/*plan_comment'))
    if paths:
        monaco_approval_comments[patient_id] = paths

In [ ]:
monaco_approval_comments

In [ ]:
patient_ids_with_approved_plans = list(monaco_approval_comments.keys())

In [ ]:
tel_files = {}

for patient_id in patient_ids_with_approved_plans:
    paths = list(monaco_directory.glob(f'*~{patient_id}/plan/*/tel.1'))
    if paths:
        tel_files[patient_id] = paths

In [ ]:
tel_files

In [ ]:
patient_ids_to_check = list(tel_files.keys())

In [ ]:
def run_for_a_patient_id(patient_id):
    markdown_print(f"# {patient_id}")
    
    for tel_file in tel_files[patient_id]:
        markdown_print(f"## `{tel_file}`")
        
        try:
            mudensity_tel = get_mu_density_from_file(tel_file)

            for trf_file in logfile_paths[patient_id]:
                markdown_print(f"### `{trf_file}`")

                mudensity_trf = get_mu_density_from_file(trf_file)

                gamma = calc_gamma(to_tuple(mudensity_tel), to_tuple(mudensity_trf))

                results_dir = output_directory.joinpath(patient_id, tel_file.parent.name, trf_file.stem)
                results_dir.mkdir(exist_ok=True, parents=True)
                
                header_text = (
                    f"Patient ID: {patient_id}\n"
                    f"Plan Name: {tel_file.parent.name}\n"
                )
                
                footer_text = (
                    f"tel.1 file path: {str(tel_file)}\n"
                    f"trf file path: {str(trf_file)}\n"
                    f"results path: {str(results_dir)}"
                )
                
                png_filepath = str(results_dir.joinpath("result.png").resolve())
                pdf_filepath = str(pdf_directory.joinpath(f"{patient_id}.pdf").resolve())
                
                fig = plot_and_save_results(
                    mudensity_tel, mudensity_trf, 
                    gamma, png_filepath, pdf_filepath, 
                    header_text=header_text, footer_text=footer_text
                )
                
                fig.tight_layout()
                plt.savefig(png_filepath, dpi=300)
                plt.show()

                !magick convert "{png_filepath}" "{pdf_filepath}"
                
        
        except Exception as e:
            traceback.print_exc()


for patient_id in patient_ids_to_check:
    run_for_a_patient_id(patient_id)